# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,queenstown,-31.8976,26.8753,18.26,82,13,2.29,ZA,1736485003
1,1,port elizabeth,-33.9180,25.5701,22.40,84,40,3.09,ZA,1736485005
2,2,bilibino,68.0546,166.4372,-27.35,99,100,1.07,RU,1736485006
3,3,tura,25.5198,90.2201,22.87,43,0,1.34,IN,1736485007
4,4,margaret river,-33.9500,115.0667,21.77,59,100,5.25,AU,1736485009


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

import hvplot.pandas
import geoviews as gv
import geopandas as gpd
from geoviews import opts

# Convert city_data_df to a GeoDataFrame
city_gdf = gpd.GeoDataFrame(
    city_data_df, geometry=gpd.points_from_xy(city_data_df["Lng"], city_data_df["Lat"])
)

# Configure the map plot
humidity_map = city_gdf.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", size="Humidity", color="blue",
    hover_cols=["City", "Country", "Humidity"]
).opts(title="City Locations with Humidity Levels")

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit the ideal weather criteria
ideal_vacation_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_vacation_df = ideal_vacation_df.dropna()

# Display sample data
ideal_vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,tura,25.5198,90.2201,22.87,43,0,1.34,IN,1736485007
67,67,san sebastian de la gomera,28.0916,-17.1133,21.20,22,0,1.03,ES,1736485138
135,135,east london,-33.0153,27.9116,21.52,73,0,3.09,ZA,1736485229
336,336,baiti,13.7450,-14.7482,23.18,19,0,2.24,SN,1736485529
352,352,ingeniero guillermo n. juarez,-23.9000,-61.8500,25.30,41,0,2.15,AR,1736485551


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,tura,IN,25.5198,90.2201,43,
67,san sebastian de la gomera,ES,28.0916,-17.1133,22,
135,east london,ZA,-33.0153,27.9116,73,
336,baiti,SN,13.7450,-14.7482,19,
352,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
import requests
from api_keys import geoapify_key  # Ensure your Geoapify API key is stored in api_keys.py

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tura - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
baiti - nearest hotel: No hotel found
ingeniero guillermo n. juarez - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
3,tura,IN,25.5198,90.2201,43,No hotel found
67,san sebastian de la gomera,ES,28.0916,-17.1133,22,No hotel found
135,east london,ZA,-33.0153,27.9116,73,No hotel found
336,baiti,SN,13.7450,-14.7482,19,No hotel found
352,ingeniero guillermo n. juarez,AR,-23.9000,-61.8500,41,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Convert hotel_df to a GeoDataFrame
hotel_gdf = gpd.GeoDataFrame(
    hotel_df, geometry=gpd.points_from_xy(hotel_df["Lng"], hotel_df["Lat"])
)

# Configure the map plot with hotel locations and additional hover information
hotel_map = hotel_gdf.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM", size=100, color="red",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
).opts(title="Ideal Vacation Locations with Nearby Hotels")

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)